In [19]:
# Código para consulta de quais budgets estão criados a partir do tenancy especificado no arquivo de configuracao 

import oci
import sys

# Default config file and profile
config = oci.config.from_file(file_location="C:\\Users\\rodrigo.almeida\\Documents\\GitHub\\BudgetReports\\.oci\\config")
budget_client = oci.budget.BudgetClient(config)

print(config)

# The first argument is the name of the script, so start the index at 1
compartment_id = config['tenancy']


print("Compartment Id: " + compartment_id)


# list all budgets
budgets = oci.pagination.list_call_get_all_results(
    budget_client.list_budgets,
    compartment_id
).data
print('ListBudgets for compartment with OCID: {}'.format(compartment_id))
for budget in budgets:
    print(budget)


{'log_requests': False, 'additional_user_agent': '', 'pass_phrase': None, 'user': 'ocid1.user.oc1..aaaaaaaa3nuy5edskgekop7jbessyg2ynnts3rrlteivjmwuzbdjey5upobq', 'fingerprint': 'ed:d0:c3:4f:35:98:6c:87:7c:e9:ff:5b:c4:36:af:96', 'tenancy': 'ocid1.tenancy.oc1..aaaaaaaatibiivijlqznqwwgrzxbx7ccsf4o4oxowqazkds7qurhtus6r6sa', 'region': 'sa-saopaulo-1', 'key_file': 'C:\\Users\\rodrigo.almeida\\Documents\\GitHub\\BudgetReports\\rodrigo.almeida@dataprev.gov.br_2024-12-04T14_35_50.229Z.pem'}
Compartment Id: ocid1.tenancy.oc1..aaaaaaaatibiivijlqznqwwgrzxbx7ccsf4o4oxowqazkds7qurhtus6r6sa
ListBudgets for compartment with OCID: ocid1.tenancy.oc1..aaaaaaaatibiivijlqznqwwgrzxbx7ccsf4o4oxowqazkds7qurhtus6r6sa
{
  "actual_spend": 384.686293949584,
  "alert_rule_count": 7,
  "amount": 3527.0,
  "budget_processing_period_start_offset": 1,
  "compartment_id": "ocid1.tenancy.oc1..aaaaaaaatibiivijlqznqwwgrzxbx7ccsf4o4oxowqazkds7qurhtus6r6sa",
  "defined_tags": {
    "Oracle-Tags": {
      "CreatedBy": "defau

In [ ]:
# Antes de executar o codigo, gerar os arquivos usage_special.csv e usage_compartment.csv
# python showusage.py -ds 2025-02-05  -report SPECIAL -t ALMEIDA -csv 
# python showusage.py -ds 2025-02-05  -report COMPARTMENT -t ALMEIDA -csv 
### ANTES DE EXECUTAR GARANTIR QUE O BANCO NEO4J ESTÁ NO AR ###

import seaborn as sns
import pandas as pd
from neo4j import GraphDatabase
import hashlib
import os

# Data da Consulta
date = "07/02/2025"

def computeMD5hash(my_string):
    m = hashlib.md5()
    m.update(my_string.encode('utf-8'))
    return m.hexdigest()

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j://localhost"
AUTH = ("neo4j", "vini1804")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()


driver = GraphDatabase.driver(URI, auth=AUTH)
session = driver.session(database="neo4j")

file_compartment = "usage_compartments.csv"
file_special = "usage_special.csv"
file_name = "dados.csv"
df_compartment = pd.read_csv(file_compartment)

df_special = pd.read_csv(file_special)

df_compartment = df_compartment[["Compartment Path", "Service", "Currency", "Cost"]] # as duas primeiras colunas do dataframe

df_special = df_special.iloc[:, 0:4] # as quatro primeiras colunas do dataframe

df_compartment.set_index('Service')
df_special.set_index('Service')

df = df_special.merge(df_compartment, on='Service', how='left')



query = '''
unwind $data as row
MERGE (d:Date {date:row.Date})
MERGE (co:Cost {cost:row.Cost, currency: row.Currency, Cost_id: row.Cost_id})
MERGE (c:Compartment {name: row.Compartment_Path})
MERGE (s:Service {name: row.Service})
MERGE (r:Region {name: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Product_Name})
MERGE (s)-[:AVAILABLE_IN]->(r)
MERGE (s)-[:OFFERS]->(p)
MERGE (p)-[:CONSUMED_BY]->(c)
MERGE (c)-[:COSTS]->(co)
MERGE (p)-[:COSTS]->(co)
MERGE (co)-[:IN]->(d)
'''

query2 = '''
unwind $data as row
MERGE (c:Compartment {name: row.Compartment_Path, regiao: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Product_Name, servico: row.Service})
MERGE (c)-[:CONSUMIU {cost: row.Cost, date:row.Date}]->(p)

'''

df.to_csv(file_name, sep=';', encoding='utf-8', index=False, header=True)

for key, value in df.iterrows():
    data = {
        'Service': value['Service'],
        'Region': value['Region'],
        'Product_SKU': value['Product SKU'],
        'Product_Name': value['Product Name'],
        'Compartment_Path': value['Compartment Path'],
        'Currency': value['Currency'],
        'Cost': value['Cost'],
        'Cost_id': computeMD5hash(value['Product SKU'] + value['Compartment Path']),
        'Date': date
    }
    records, summary, keys = driver.execute_query(query2, data=data, )
    print("The query `{query}` returned {records_count} records in {time} ms.".format(query=summary.query, records_count=len(records),time=summary.result_available_after, ))



# session/driver usage

session.close()
driver.close()





The query `
unwind $data as row
MERGE (c:Compartment {name: row.Compartment_Path, regiao: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Product_Name, servico: row.Service})
MERGE (c)-[:CONSUMIU {cost: row.Cost, date:row.Date}]->(p)

` returned 0 records in 49 ms.
The query `
unwind $data as row
MERGE (c:Compartment {name: row.Compartment_Path, regiao: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Product_Name, servico: row.Service})
MERGE (c)-[:CONSUMIU {cost: row.Cost, date:row.Date}]->(p)

` returned 0 records in 1 ms.
The query `
unwind $data as row
MERGE (c:Compartment {name: row.Compartment_Path, regiao: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Product_Name, servico: row.Service})
MERGE (c)-[:CONSUMIU {cost: row.Cost, date:row.Date}]->(p)

` returned 0 records in 1 ms.
The query `
unwind $data as row
MERGE (c:Compartment {name: row.Compartment_Path, regiao: row.Region})
MERGE (p:Product {sku: row.Product_SKU, name: row.Produc

In [21]:
import os

from graphdatascience import GraphDataScience
import pandas as pd

NEO4J_AUTH = None
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")
NEO4J_DB = os.environ.get("NEO4J_DB", "neo4j")


if os.environ.get("NEO4J_USER") and os.environ.get("NEO4J_PASSWORD"):
    NEO4J_AUTH = (
        os.environ.get("neo4j"),
        os.environ.get("vini1804"),
    )

gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH, database=NEO4J_DB)


## Configurando modelo: https://neo4j.com/docs/graph-data-science-client/current/model-object/

G, result = gds.graph.project("Compartment", "Product", "CONSUMIU")

print(f"The projection took {result['projectMillis']} ms")

# We can use convenience methods on `G` to check if the projection looks correct
print(f"Graph '{G.name()}' node count: {G.node_count()}")
print(f"Graph '{G.name()}' node labels: {G.node_labels()}")
print(f"Graph '{G.name()}' relationship count: {G.relationship_count()}")


The projection took 10 ms
Graph 'Compartment' node count: 19
Graph 'Compartment' node labels: ['Product']
Graph 'Compartment' relationship count: 0
